<a href="https://colab.research.google.com/github/codigoquant/python_para_investimentos/blob/master/16_CVM_Os_Melhores_e_os_Piores_Fundos_de_Investimento_do_mes_Python_para_Investimentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Ricos pelo Acaso

*   Link para o vídeo: https://youtu.be/NHCUUZOvk7k
---
*   Base de Dados: http://dados.cvm.gov.br/



###Coletando os dados da CVM

In [1]:
import pandas as pd
pd.set_option("display.max_colwidth", 150)
#pd.options.display.float_format = '{:.2f}'.format

>Funções que buscam dados no site da CVM e retornam um DataFrame Pandas:


In [2]:
def busca_informes_cvm(ano, mes):
  url = 'http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{:02d}{:02d}.csv'.format(ano,mes)
  return pd.read_csv(url, sep=';')

In [3]:
def busca_cadastro_cvm():
  url = "http://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv"
  return pd.read_csv(url, sep=';', encoding='ISO-8859-1')

>Buscando dados no site da CVM

In [4]:
informes_diarios = busca_informes_cvm(2021,9)

In [5]:
informes_diarios

,TP_FUNDO,CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST
0,FI,00.017.024/0001-53,2021-09-01,1068841.27,27.647912,1074030.80,0.0,0.0,1
1,FI,00.017.024/0001-53,2021-09-02,1069062.59,27.653245,1074237.97,0.0,0.0,1
2,FI,00.017.024/0001-53,2021-09-03,1069334.42,27.656700,1074372.20,0.0,0.0,1
3,FI,00.017.024/0001-53,2021-09-06,1069574.10,27.661456,1074556.94,0.0,0.0,1
4,FI,00.017.024/0001-53,2021-09-08,1069477.32,27.665384,1074709.52,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...
460012,FI,97.929.213/0001-34,2021-09-24,70514866.08,9.469475,71961584.37,0.0,0.0,2
460013,FI,97.929.213/0001-34,2021-09-27,71851360.45,9.453396,71839395.62,0.0,0.0,2
460014,FI,97.929.213/0001-34,2021-09-28,71313013.11,9.382502,71300646.29,0.0,0.0,2
460015,FI,97.929.213/0001-34,2021-09-29,71344489.38,9.386591,71331720.43,0.0,0.0,2


In [6]:
cadastro_cvm = busca_cadastro_cvm()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [7]:
cadastro_cvm

,TP_FUNDO,CNPJ_FUNDO,DENOM_SOCIAL,DT_REG,DT_CONST,CD_CVM,DT_CANCEL,SIT,DT_INI_SIT,DT_INI_ATIV,DT_INI_EXERC,DT_FIM_EXERC,CLASSE,DT_INI_CLASSE,RENTAB_FUNDO,CONDOM,FUNDO_COTAS,FUNDO_EXCLUSIVO,TRIB_LPRAZO,INVEST_QUALIF,INVEST_PROF,ENTID_INVEST,TAXA_PERFM,INF_TAXA_PERFM,TAXA_ADM,INF_TAXA_ADM,VL_PATRIM_LIQ,DT_PATRIM_LIQ,DIRETOR,CNPJ_ADMIN,ADMIN,PF_PJ_GESTOR,CPF_CNPJ_GESTOR,GESTOR,CNPJ_AUDITOR,AUDITOR,CNPJ_CUSTODIANTE,CUSTODIANTE,CNPJ_CONTROLADOR,CONTROLADOR
0,FACFIF,00.000.684/0001-21,DEUTSCHE BANK FDO APLIC QUOTAS FDO INV FINANCEIRO - MAX,2003-04-30,1994-12-20,19.0,2000-08-01,CANCELADA,2000-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FACFIF,00.000.731/0001-37,ITAMARITI CASH FUNDO APLICACAO QUOTAS FDOS INVESTIMENTO,2003-04-30,1994-05-18,40681.0,1996-01-26,CANCELADA,1996-01-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,FACFIF,00.000.732/0001-81,FUNDO APLIC. QUOTAS DE F.I. SANTANDER CURTO PRAZO,2003-04-30,1994-05-24,27.0,1999-09-03,CANCELADA,1999-09-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FACFIF,00.000.740/0001-28,FUNDO DE APLIC EM QUOTAS DE FUNDOS DE INV BMC 60 DIAS URV,2003-04-30,1994-05-23,40690.0,1996-06-10,CANCELADA,1996-06-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FACFIF,00.000.749/0001-39,BALANCE FUNDO APLICACAO QUOTAS FUNDO INVESTIMENTO FINANCEIRO,2003-04-30,1994-05-12,35.0,2000-06-26,CANCELADA,2000-06-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64191,FUNCINE,28.849.405/0001-18,BTG PACTUAL INVESTIMAGE 1 - FUNDO DE FINANCIAMENTO DA INDUSTRIA CINEMATOGRAFICA NACIONAL - FUNCINE,2017-10-16,2017-10-09,417002.0,NaN,EM FUNCIONAMENTO NORMAL,2017-12-27,2017-12-27,2021-04-01,2022-03-31,FUNCINE,2017-10-09,NaN,Fechado,N,NaN,NaN,S,NaN,NaN,NaN,NaN,NaN,NaN,1661427.08,2021-06-30,NaN,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM,PJ,09.544.103/0001-97,INVESTIMAGE ADMINISTRADORA DE RECURSOS LTDA.,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S,30.306.294/0001-45,BANCO BTG PACTUAL S/A,NaN,NaN
64192,FUNCINE,28.850.140/0001-78,FUNDO DE FINANCIAMENTO DA INDUSTRIA CINEMATOGRAFICA NACIONAL - FUNCINE - TIM,2017-10-16,2017-10-06,417001.0,NaN,EM FUNCIONAMENTO NORMAL,2017-12-26,2017-12-26,2021-04-01,2022-03-31,FUNCINE,2017-10-06,NaN,Fechado,N,NaN,NaN,S,NaN,NaN,NaN,NaN,NaN,NaN,6995561.86,2021-06-30,NaN,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM,PJ,09.544.103/0001-97,INVESTIMAGE ADMINISTRADORA DE RECURSOS LTDA.,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S,30.306.294/0001-45,BANCO BTG PACTUAL S/A,NaN,NaN
64193,FUNCINE,28.912.043/0001-62,HAPVIDA - FUNDO DE FINANCIAMENTO DA INDUSTRIA CINEMATOGRAFICA NACIONAL - FUNCINE,2017-10-27,2017-10-19,417003.0,NaN,EM FUNCIONAMENTO NORMAL,2018-12-21,2018-12-21,2021-04-01,2022-03-31,FUNCINE,2017-10-19,NaN,Fechado,N,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,5086522.11,2021-06-30,NaN,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM,PJ,09.544.103/0001-97,INVESTIMAGE ADMINISTRADORA DE RECURSOS LTDA.,61.562.112/0001-20,PRICEWATERHOUSECOOPERS AUDITORES INDEPENDENTES,30.306.294/0001-45,BANCO BTG PACTUAL S/A,NaN,NaN
64194,FUNCINE,31.674.280/0001-47,BTG PACTUAL INVESTIMAGE III - FUNDO DE FINANCIAMENTO DA INDUSTRIA CINEMATOGRAFICA NACIONAL - FUNCINE,2018-10-26,2018-09-13,418001.0,NaN,FASE PRÉ-OPERACIONAL,2018-10-26,2018-10-26,NaN,NaN,FUNCINE,2018-09-13,NaN,Fechado,N,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM,PJ,09.544.103/0

In [ ]:
cadastro_cvm

,CNPJ_FUNDO,DENOM_SOCIAL,DT_REG,DT_CONST,DT_CANCEL,SIT,DT_INI_SIT,DT_INI_ATIV,DT_INI_EXERC,DT_FIM_EXERC,CLASSE,DT_INI_CLASSE,RENTAB_FUNDO,CONDOM,FUNDO_COTAS,FUNDO_EXCLUSIVO,TRIB_LPRAZO,INVEST_QUALIF,TAXA_PERFM,INF_TAXA_PERFM,TAXA_ADM,INF_TAXA_ADM,VL_PATRIM_LIQ,DT_PATRIM_LIQ,DIRETOR,CNPJ_ADMIN,ADMIN,PF_PJ_GESTOR,CPF_CNPJ_GESTOR,GESTOR,CNPJ_AUDITOR,AUDITOR,CNPJ_CUSTODIANTE,CUSTODIANTE,CNPJ_CONTROLADOR,CONTROLADOR
0,36.498.416/0001-29,03 FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO,2020-04-09,2020-04-09,NaN,EM FUNCIONAMENTO NORMAL,2020-04-27,2020-04-27,2020-04-27,2020-06-30,Fundo Multimercado,2020-04-09,NaN,Aberto,S,N,S,N,NaN,NaN,NaN,NaN,1.500212e+08,2020-04-29,ALLAN HADID,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM,PJ,09.428.261/0001-81,O3 GESTÃO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S,30.306.294/0001-45,BANCO BTG PACTUAL S/A,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM
1,36.729.776/0001-94,03 MASTER FUNDO DE INVESTIMENTO EM ACOES,2020-04-08,2020-04-08,NaN,FASE PRÉ-OPERACIONAL,2020-04-08,NaN,NaN,NaN,Fundo de Ações,2020-04-08,NaN,Aberto,N,N,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,ALLAN HADID,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM,PJ,09.428.261/0001-81,O3 GESTÃO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S,30.306.294/0001-45,BANCO BTG PACTUAL S/A,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM
2,36.499.643/0001-79,03 MASTER FUNDO DE INVESTIMENTO MULTIMERCADO,2020-04-09,2020-04-09,NaN,EM FUNCIONAMENTO NORMAL,2020-04-27,2020-04-27,2020-04-27,2020-06-30,Fundo Multimercado,2020-04-09,NaN,Aberto,N,N,S,N,NaN,NaN,NaN,NaN,1.498389e+08,2020-04-29,ALLAN HADID,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM,PJ,09.428.261/0001-81,O3 GESTÃO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S,30.306.294/0001-45,BANCO BTG PACTUAL S/A,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM
3,36.016.411/0001-12,051 ALLOCATION FUNDO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO CREDITO PRIVADO,2020-04-03,2020-04-02,NaN,FASE PRÉ-OPERACIONAL,2020-04-02,NaN,NaN,NaN,Fundo Multimercado,2020-04-02,NaN,Aberto,S,N,S,S,NaN,NaN,NaN,NaN,NaN,NaN,ALLAN HADID,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM,PJ,30.193.705/0001-33,ZERO CINCO UM CAPITAL GESTÃO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S,30.306.294/0001-45,BANCO BTG PACTUAL S/A,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM
4,33.913.620/0001-70,051 BRUCUTU FUNDO DE INVESTIMENTO MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO EXTERIOR,2019-07-19,2019-06-04,NaN,EM FUNCIONAMENTO NORMAL,2019-08-05,2019-08-05,2019-08-05,2020-05-31,Fundo Multimercado,2019-06-04,DI de um dia,Fechado,N,S,NaN,S,20.0,NaN,0.2,NaN,2.442650e+07,2020-04-29,MARCELO VIEIRA FRANCISCO,62.318.407/0001-19,SANTANDER CACEIS BRASIL DISTRIBUIDORA DE TITULOS E VALORES MOBILIARIOS S.A,PJ,30.193.705/0001-33,ZERO CINCO UM CAPITAL GESTÃO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S,62.318.407/0001-19,SANTANDER CACEIS BRASIL DISTRIBUIDORA DE TITULOS E VALORES MOBILIARIOS S.A,62.318.407/0001-19,SANTANDER CACEIS BRASIL DISTRIBUIDORA DE TITULOS E VALORES MOBILIARIOS S.A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36770,31.340.900/0001-01,ZURICH VALORA PREVIDENCIARIO FI EM COTAS DE FUNDOS DE INVESTIMENTO RENDA FIXA CREDITO PRIVADO,2018-12-19,2018-12-19,NaN,EM FUNCIONAMENTO NORMAL,2019-03-27,2019-03-27,2020-01-01,2020-12-31,Fundo de Renda Fixa,2018-12-19,NaN,Aberto,S,S,N,S,20.0,O fundo cobrara 20.00 % do que exceder 100.00 % o indice CDIE. A taxa de performance sera paga no(s) mes(es) de Janeiro e Julho,1.1,NaN,2.084469e+07,2020-04-29,ALLAN HADID,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM,PJ,07.559.989/0001-17,VALORA GESTÃO DE INVESTIMENTOS LTDA.,61

###Manipulando os dados da CVM

>Definindo filtros para os Fundos de Investimento


In [8]:
minimo_cotistas = 100

>Manipulando os dados e aplicando filtros

In [9]:
fundos = informes_diarios[informes_diarios['NR_COTST'] >= minimo_cotistas].pivot(index='DT_COMPTC', columns='CNPJ_FUNDO', values=['VL_TOTAL',	'VL_QUOTA',	'VL_PATRIM_LIQ',	'CAPTC_DIA',	'RESG_DIA'])

In [10]:
fundos

VL_TOTAL  ...           RESG_DIA
CNPJ_FUNDO 00.068.305/0001-35  ... 97.519.794/0001-36
DT_COMPTC                      ...                   
2021-09-01        40107139.12  ...           53689.52
2021-09-02        40109119.37  ...          110077.27
2021-09-03        40111191.76  ...          309838.48
2021-09-06        40072933.70  ...          106344.35
2021-09-08        40047057.99  ...           46336.49
2021-09-09        40051124.38  ...          144967.73
2021-09-10        39978797.84  ...          472360.38
2021-09-13        39979158.58  ...          120038.03
2021-09-14        39977797.13  ...           88882.56
2021-09-15        39958635.51  ...           92373.71
2021-09-16        39963236.50  ...          491379.66
2021-09-17        39936230.98  ...          186744.45
2021-09-20        39944964.90  ...          419805.11
2021-09-21        39758089.10  ...          316612.66
2021-09-22        39768112.53  ...          383626.72
2021-09-23        39711174.22  ...          117746.22
2021-09-24        39698330.60  ...           37446.35
2021-09-27        39686798.93  ...           28980.91
2021-09-28        39677083.12  ...          325314.57
2021-09-29        39700621.58  ...           57416.65
2021-09-30        39696582.78  ...          156555.17

[21 rows x 17795 columns]

>Normalizando os dados de cotas para efeitos comparativos

In [11]:
cotas_normalizadas = fundos['VL_QUOTA'] / fundos['VL_QUOTA'].iloc[0]

In [12]:
cotas_normalizadas

CNPJ_FUNDO,00.068.305/0001-35,00.071.477/0001-68,00.073.041/0001-08,00.089.915/0001-15,00.180.995/0001-10,00.185.259/0001-54,00.222.725/0001-24,00.222.816/0001-60,00.280.302/0001-60,00.306.278/0001-91,00.322.699/0001-06,00.346.750/0001-10,00.360.293/0001-18,00.398.561/0001-90,00.400.490/0001-13,00.524.617/0001-06,00.539.553/0001-17,00.575.922/0001-27,00.598.452/0001-17,00.601.692/0001-23,00.743.026/0001-20,00.756.851/0001-69,00.777.815/0001-81,00.785.365/0001-79,00.793.947/0001-05,00.807.777/0001-62,00.812.433/0001-41,00.813.340/0001-31,00.813.342/0001-20,00.817.201/0001-86,00.817.677/0001-17,00.819.855/0001-49,00.819.858/0001-82,00.819.880/0001-22,00.819.889/0001-33,00.819.901/0001-00,00.819.915/0001-23,00.819.929/0001-47,00.822.059/0001-65,00.822.954/0001-80,...,43.279.729/0001-51,47.176.334/0001-84,47.177.431/0001-91,47.177.894/0001-53,47.178.058/0001-93,47.178.215/0001-60,47.178.405/0001-88,47.179.288/0001-77,53.633.558/0001-70,54.069.422/0001-42,54.486.055/0001-82,55.075.238/0001-78,55.234.165/0001-10,56.573.470/0001-07,56.824.857/0001-80,60.906.179/0001-72,63.375.216/0001-51,67.845.909/0001-11,67.845.925/0001-04,67.976.423/0001-12,67.976.449/0001-60,68.599.141/0001-06,68.623.479/0001-56,68.670.512/0001-07,68.971.183/0001-26,70.951.678/0001-35,73.232.530/0001-39,73.899.759/0001-21,73.970.642/0001-97,74.055.922/0001-32,74.326.471/0001-20,77.054.658/0001-00,88.002.696/0001-36,88.198.056/0001-43,96.498.654/0001-66,96.498.985/0001-04,96.500.715/0001-82,97.261.093/0001-40,97.519.703/0001-62,97.519.794/0001-36
DT_COMPTC,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-09-01,1.000000,1.000000,1.000000,1.000000,1.000000,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2021-09-02,1.000197,1.000124,1.000168,1.000149,0.999918,NaN,0.999987,0.999915,1.000053,1.000211,1.000234,0.999656,1.003586,0.986450,0.999897,1.003111,1.000224,0.975612,1.000190,0.979782,1.000154,1.000169,1.000228,1.000170,1.000204,1.000217,1.000117,1.000073,1.000208,1.000170,1.000230,1.002947,0.999404,1.003068,1.003072,1.000159,1.000185,1.000164,0.976811,1.003155,...,NaN,0.978029,0.975367,0.977993,0.977920,0.978929,0.978019,0.977349,0.978314,0.980899,0.979914,0.976714,0.980016,0.979927,0.975612,1.000215,0.981295,0.982872,0.977988,0.978433,0.977211,1.000143,1.000197,0.983861,1.000240,0.982242,0.982424,0.977352,1.000222,0.978379,1.003838,0.981371,0.978159,0.978742,0.980968,0.977102,1.000246,0.977611,1.000303,1.000241
2021-09-03,1.000327,1.000247,1.000335,1.000300,1.000168,NaN,1.000190,1.000158,1.000231,1.000399,1.000436,0.999885,1.002870,0.980759,0.999527,1.002976,1.000434,0.981943,1.000414,0.983239,1.000340,1.000281,1.000409,1.000328,1.000247,1.000429,1.000238,1.000126,1.000402,1.000327,1.000402,1.002726,0.999361,1.002959,1.002967,1.000451,1.000501,1.000460,0.977117,1.003037,...,NaN,0.980898,0.981406,0.980874,0.980802,0.983844,0.980892,0.979213,0.982355,0.979158,0.983521,0.977041,0.983728,0.983527,0.981665,1.000407,0.982588,0.983053,0.980315,0.982563,0.979230,1.000311,1.000328,0.997950,1.000483,0.980956,0.984190,0.979118,1.000414,0.982461,1.001017,0.984275,0.978381,0.976254,0.979466,0.979084,1.000442,0.979941,1.000574,1.000485
2021-09-06,1.000496,1.000372,1.000503,1.000445,1.000458,NaN,1.000498,1.000469,1.000468,1.000606,1.000660,1.000392,1.001630,0.983236,0.999807,1.000751,1.000676,0.986640,1.000605,0.992073,1.000494,1.00

###Fundos de Investimento com os melhores desempenhos em Abril de 2020

In [13]:
melhores = pd.DataFrame()
melhores['retorno(%)'] = (cotas_normalizadas.iloc[-1].sort_values(ascending=False)[:5] - 1) * 100
melhores

,retorno(%)
CNPJ_FUNDO,
31.713.505/0001-27,17.335507
40.503.169/0001-89,17.281894
28.971.507/0001-01,16.518635
40.920.005/0001-57,16.498366
40.936.530/0001-60,16.485573


>Buscando dados dos Fundos de Investimento pelo CNPJ

In [14]:
for cnpj in melhores.index:
  fundo = cadastro_cvm[cadastro_cvm['CNPJ_FUNDO'] == cnpj]
  melhores.at[cnpj, 'Fundo de Investimento'] = fundo['DENOM_SOCIAL'].values[0]
  melhores.at[cnpj, 'Classe'] = fundo['CLASSE'].values[0]
  melhores.at[cnpj, 'PL'] = fundo['VL_PATRIM_LIQ'].values[0]

In [15]:
melhores

,retorno(%),Fundo de Investimento,Classe,PL
CNPJ_FUNDO,,,,
31.713.505/0001-27,17.335507,CSHG PÁTRIA INF IV FUNDO DE INVESTIMENTO MULTIMERCADO,Fundo Multimercado,2.070111e+08
40.503.169/0001-89,17.281894,VITREO URANIO FUNDO DE INVESTIMENTO MULTIMERCADO,Fundo Multimercado,9.019566e+07
28.971.507/0001-01,16.518635,CSHG PÁTRIA PRIVATE EQUITY VI FUNDO DE INVESTIMENTO MULTIMERCADO,Fundo Multimercado,2.044490e+08
40.920.005/0001-57,16.498366,VISTA MULTIESTRATEGIA ADVISORY FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO,Fundo Multimercado,3.691510e+08
40.936.530/0001-60,16.485573,VITREO PETRÓLEO FUNDO DE INVESTIMENTO MULTIMERCADO,Fundo Multimercado,8.498524e+06


###Fundos de Investimento com os piores desempenhos em Abril de 2020

In [16]:
piores = pd.DataFrame()
piores['retorno(%)'] = (cotas_normalizadas.iloc[-1].sort_values(ascending=True)[:5] - 1) * 100
piores

,retorno(%)
CNPJ_FUNDO,
24.837.071/0001-01,-38.064958
20.468.448/0001-41,-32.059960
30.338.659/0001-13,-25.240942
13.199.100/0001-30,-23.220382
10.292.302/0001-34,-23.176476


>Buscando dados dos Fundos de Investimento pelo CNPJ

In [17]:
for cnpj in piores.index:
  fundo = cadastro_cvm[cadastro_cvm['CNPJ_FUNDO'] == cnpj]
  piores.at[cnpj, 'Fundo de Investimento'] = fundo['DENOM_SOCIAL'].values[0]
  piores.at[cnpj, 'Classe'] = fundo['CLASSE'].values[0]
  piores.at[cnpj, 'PL'] = fundo['VL_PATRIM_LIQ'].values[0]

In [18]:
piores

,retorno(%),Fundo de Investimento,Classe,PL
CNPJ_FUNDO,,,,
24.837.071/0001-01,-38.064958,WM DELTA CRÉDITO ESTRUTURADO FI EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO - CRÉDITO PRIVADO,Fundo Multimercado,6.038072e+05
20.468.448/0001-41,-32.059960,CSHG JIVE DISTRESSED ALLOCATION FI MULTIMERCADO CRÉDITO PRIVADO IE,Fundo Multimercado,1.191308e+08
30.338.659/0001-13,-25.240942,CHESS ALPHA FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO,Fundo Multimercado,2.896054e+07
13.199.100/0001-30,-23.220382,SANTANDER FUNDO DE INVESTIMENTO COLABORADORES MAGAZINE LUIZA AÇÕES,Fundo de Ações,1.967289e+07
10.292.302/0001-34,-23.176476,ZENITH HAYP FUNDO DE INVESTIMENTO EM AÇÕES,Fundo de Ações,1.502293e+08
